In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [2]:
#!/bin/bash
! kaggle datasets download amananandrai/ag-news-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset
License(s): unknown
  0% 0.00/11.4M [00:00<?, ?B/s]
100% 11.4M/11.4M [00:00<00:00, 1.44GB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/ag-news-classification-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/train.csv')

In [6]:
df.shape

(120000, 3)

In [7]:
df.dropna(inplace = True)

In [8]:
df.shape

(120000, 3)

In [9]:
df = df.sample(n= 10000 , random_state = 42).reset_index(drop = True)

In [10]:
df.shape

(10000, 3)

In [51]:
df.head()

,sentiment,title,description
0,2,bbc set major shakeup claim newspaper,london british broadcasting corporation world ...
1,2,marsh averts cash crunch,embattled insurance broker 39s bank agree waiv...
2,1,jeter yankee look take control ap,ap derek jeter turned season started terrible ...
3,3,flying sun safety,genesis capsule come back earth sample sun hel...
4,2,stock seen flat nortel oil weigh,new york reuters yous stock set open near unch...


In [52]:
df['description'][0]

'london british broadcasting corporation world 39s biggest public broadcaster cut almost quarter 28 000strong workforce biggest shakeup 82year history time newspaper london said monday'

In [11]:
import nltk
import string
import re
! pip install contractions

import contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 13.1 MB/s eta 0:00:00


In [12]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [14]:
stop_word = set(stopwords.words('english'))

lemma = WordNetLemmatizer()

In [15]:
def clean_text(text):
  text = contractions.fix(text)
  text = text.lower()
  text = re.sub(r'http\S+' , '' , text)
  text = re.sub(r'<.*?>'  , '' , text)
  text = text.translate(str.maketrans('' , '' , string.punctuation))
  token = word_tokenize(text)
  final = [lemma.lemmatize(word) for word in token if word not in stop_word]
  return ' '.join(final)

In [16]:
df.columns = ['sentiment' , 'title' , 'description']

In [17]:
df['title'] = df['title'].apply(clean_text)

In [18]:
df['description'] = df['description'].apply(clean_text)

In [19]:
x = df['title'] + " " + df['description']

In [20]:
df['sentiment'].value_counts()

,count
sentiment,
2,2549
4,2515
3,2471
1,2465


In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
le = LabelEncoder(

)

In [23]:
df['sentiment'] = le.fit_transform(df['sentiment'])

In [24]:
df['sentiment'].value_counts()

,count
sentiment,
1,2549
3,2515
2,2471
0,2465


In [25]:
y = df['sentiment']

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
x_train , x_test , y_train , y_test = train_test_split(x , y, test_size = 0.2 , random_state = 42)

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
token = Tokenizer(num_words = 4000 , oov_token = 'OOV')

In [30]:
token.fit_on_texts(x_train)

In [31]:
train_seq = token.texts_to_sequences(x_train)
test_seq = token.texts_to_sequences(x_test)

In [32]:
import joblib

joblib.dump(token , 'ag_token.pkl')

['ag_token.pkl']

In [33]:
max_len = max([len(seq) for seq in train_seq])

In [34]:
max_len

98

In [35]:
train_padding = pad_sequences(train_seq , maxlen = max_len , padding = 'post')

In [36]:
test_padding = pad_sequences(test_seq , maxlen = max_len , padding = 'post')

In [37]:
train_padding[0]

array([  42,   43, 1073,    1,    1,  439,   42,   43,   93,  718,  745,
       1317,    1,  487, 3255,    9,    1, 1465,  277,  662, 3525,   85,
       2851,   98,  344,  594,  243,    1,    4,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [38]:
from keras.models import *
from keras.layers import *

In [39]:
input = Input(shape=(max_len,))

emb = Embedding(4000, 32, mask_zero=True)(input)

d1 = Bidirectional(LSTM(64))(emb)

d2 = Dense(64, activation='relu')(d1)

d3 = Dropout(0.4)(d2)

output = Dense(4, activation='softmax')(d3)

model = Model(input, output)


In [40]:
model.summary(

)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 98)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 98, 32)    │    128,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 98)        │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128)       │     49,664 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      8,256 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 4)         │        260 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 186,180 (727.27 KB)

 Trainable params: 186,180 (727.27 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
from keras.optimizers import *
from keras.losses import *
from keras.callbacks import *

In [42]:
model.compile(optimizer = Adam(1e-3) , loss = SparseCategoricalCrossentropy(from_logits = False), metrics = ['accuracy'])

In [43]:
es = EarlyStopping(monitor = 'val_loss' , mode = 'min'  , restore_best_weights = True , patience = 3)

In [44]:
ls = ReduceLROnPlateau(monitor = 'val_loss' , patience = 3 , factor = 0.4)

In [45]:
model.fit(train_padding , y_train , validation_data = (test_padding , y_test) , epochs = 10 , batch_size = 16 , verbose = 1 , callbacks = [es , ls])

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.4852 - loss: 1.0628 - val_accuracy: 0.8565 - val_loss: 0.4266 - learning_rate: 0.0010
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.9109 - loss: 0.3078 - val_accuracy: 0.8695 - val_loss: 0.4346 - learning_rate: 0.0010
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9439 - loss: 0.2017 - val_accuracy: 0.8620 - val_loss: 0.4247 - learning_rate: 0.0010
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9681 - loss: 0.1348 - val_accuracy: 0.8605 - val_loss: 0.4764 - learning_rate: 0.0010
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9733 - loss: 0.0935 - val_accuracy: 0.8570 - val_loss: 0.5345 - learning_rate: 0.0010
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9826 - loss: 0.0638 - val_accuracy: 0.8495 - val_loss: 0.7045 - learning_rate: 0.0010


In [46]:
model.evaluate(test_padding  , y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8532 - loss: 0.4544


[0.424662321805954, 0.8619999885559082]

In [47]:
model.save('ag_new_model.keras')